In [86]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras_tuner as kt
from sklearn import metrics
import datetime
import re
import os
import seaborn as sns

In [2]:
wine_df = pd.read_csv('./WineQT.csv')

In [3]:
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [5]:
# Check for any null values
wine_df.isnull().values.any()

False

In [7]:
wine_df = wine_df.rename(columns=lambda x: re.sub('\s+', '_', x).lower())
wine_df.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'ph', 'sulphates', 'alcohol', 'quality', 'id'],
      dtype='object')

In [65]:
# Split Data Frame into x and y
x_df = wine_df[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']]
y_df = wine_df[['quality']]

In [66]:
# Turn data frames into numpy arrays for training
x_full = x_df.to_numpy()
y_full = y_df.to_numpy() - 3

In [74]:
# Scale the input values using standard scaler
x_scaler = MaxAbsScaler().fit(x_full)
x_full_scaled = x_scaler.transform(x_full)

In [75]:
# Model Builder function
def build_model(n_units, n_layers, learning_rate=.1):
    model = keras.models.Sequential()
    
    # Input layer
    model.add(keras.layers.InputLayer(input_shape=(11,), name='input_layer'))
    
    # Hidden Layers
    for i in range(n_layers):
        model.add(keras.layers.Dense(units=n_units, activation='relu', name=f'hidden_layer_{i}'))
        
    # Output Layer
    model.add(keras.layers.Dense(units=6, activation='softmax'))
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        learning_rate=learning_rate,
        metrics=['accuracy']
    )
    return model

In [76]:
def tune_and_build_model(hp):
    n_units = hp.Int('units', min_value=1, max_value=10, step=1)

    n_layers = hp.Int('layers', min_value=1, max_value=6, step=1)

    # dropout = hp.Boolean('dropout')

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=.1, sampling='log')

    # model = build_model(n_units, n_layers, learning_rate, dropout=dropout)
    model = build_model(n_units, n_layers, learning_rate)

    return model

In [77]:
# Tune with random search
CALLBACKS = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)]
tuner = kt.RandomSearch(
    hypermodel=tune_and_build_model,
    objective='val_accuracy',
    max_trials=30,
    executions_per_trial=1,
    overwrite=True,
    directory='tuning',
    project_name='wine_qt'
)
# tuner = kt.BayesianOptimization(
#     hypermodel=tune_and_build_model,
#     objective='val_accuracy',
#     max_trials=20,
#     overwrite=True,
#     directory='tuning',
#     project_name='fr_bayesian'
# )

In [78]:
# Search using all data
tuner.search(x_full_scaled, y_full, epochs=200, validation_split=.2, callbacks=CALLBACKS)

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.45414847135543823

Best val_accuracy So Far: 0.47598254680633545
Total elapsed time: 00h 00m 09s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
units             |9                 |8                 
layers            |2                 |6                 
learning_rate     |0.00010529        |0.00017927        

Train on 914 samples, validate on 229 samples
Epoch 1/200
914/914 [==============================] - 1s 754us/sample - loss: 1.7913 - accuracy: 0.0372 - val_loss: 1.7413 - val_accuracy: 0.0873
Epoch 2/200
914/914 [==============================] - 0s 118us/sample - loss: 1.7178 - accuracy: 0.2363 - val_loss: 1.6760 - val_accuracy: 0.4541
Epoch 3/200
914/914 [==============================] - 0s 93us/sample - loss: 1.6546 - accuracy: 0.4311 - val_loss: 1.5985 - val_accuracy: 0.4279
Epoch 4/200
914/914 [==============================] - 0s 92us/sample - loss: 1.5758 - accuracy: 0.4201 - val_loss

KeyboardInterrupt: 